In [ ]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer("all-MiniLM-L6-v2")

df = pd.read_csv('chunks.csv')

df['chunk_text'] = df['chunk_text'].fillna('')
texts_to_embed = df['chunk_text'].tolist()

print("Starting embedding generation...")

embeddings = model.encode(texts_to_embed, show_progress_bar=True)

print(f"Embeddings generated with shape: {embeddings.shape}")

np.save('email_embeddings.npy', embeddings)

print("Embeddings saved to email_embeddings.npy")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Starting embedding generation...


Batches:   0%|          | 0/1533 [00:00<?, ?it/s]

Embeddings generated with shape: (49033, 384)
Embeddings saved to email_embeddings.npy


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer, util

# Sample "email subjects"
subjects = [
    "Your flight booking is confirmed",
    "Flight ticket successfully booked",
    "Reminder: Dentist appointment tomorrow",
    "Latest electricity bill available online",
    "Schedule your next dental check-up"
]

### BERT embeddings
bert_model = BertModel.from_pretrained("bert-base-uncased")
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def get_bert_embeddings(sentences):
    tokens = bert_tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = bert_model(**tokens)
    # Mean pooling over token embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings

bert_embeds = get_bert_embeddings(subjects)

### Sentence Transformer embeddings
st_model = SentenceTransformer("all-MiniLM-L6-v2")
st_embeds = st_model.encode(subjects, convert_to_tensor=True)

### Compare similarity for query
query = "Book a plane ticket"
bert_query_embed = get_bert_embeddings([query])
st_query_embed = st_model.encode([query], convert_to_tensor=True)

print("\nBERT similarities:")
bert_scores = util.cos_sim(bert_query_embed, bert_embeds)
for i, score in enumerate(bert_scores[0]):
    print(f"{subjects[i]}: {score.item():.4f}")

print("\nSentence Transformer similarities:")
st_scores = util.cos_sim(st_query_embed, st_embeds)
for i, score in enumerate(st_scores[0]):
    print(f"{subjects[i]}: {score.item():.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


BERT similarities:
Your flight booking is confirmed: 0.7297
Flight ticket successfully booked: 0.5964
Reminder: Dentist appointment tomorrow: 0.5892
Latest electricity bill available online: 0.5331
Schedule your next dental check-up: 0.7168

Sentence Transformer similarities:
Your flight booking is confirmed: 0.5706
Flight ticket successfully booked: 0.6139
Reminder: Dentist appointment tomorrow: 0.0727
Latest electricity bill available online: -0.0385
Schedule your next dental check-up: 0.0971
